# A DevOps platform for AI | LangSmith for LLM Development Lifecycle
##### Learn how Langsmith enhances LLM application performance with real-time insights, tracking, and efficient AI systems, simplifying app management and ensuring key metrics are monitored and optimized

# Introduction
##### In the fast-changing world of artificial intelligence, Large Language Models (LLMs) have become vital to modern applications. They are key players in tasks like generating responses, enhancing customer interactions, and aiding in content creation. These models are central to many business operations today. But how can you ensure that your LLM app is running at its best? That's where Langsmith steps in, bringing clarity to the process.
##### In this blog, we'll look at how Langsmith helps you get the most out of your LLM app. Whether your goal is to enhance performance, fine-tune your models, or elevate user experience, Langsmith offers the tools you need to thrive in the rapidly evolving AI landscape.

# What is LangSmith?

##### Langsmith is a robust tool that enables you to monitor, track, and fine-tune the performance of your LLM or Retrieval-Augmented Generation (RAG) system. It eliminates the uncertainty of managing LLMs by offering real-time insights into how your app performs behind the scenes. Whether it's tracking metrics like response time and accuracy or identifying areas for improvement, Langsmith gives you full visibility into your app's operations.

# Need for LangSmith

##### In a typical web development project, DevOps transformed the process by promoting teamwork between software developers and IT operations. This collaboration leads to quicker updates and more dependable software, enhancing the quality of web applications and improving user experiences. Similarly, MLOps has changed the landscape of data science by providing a structured way to manage machine learning projects. It emphasizes automation, monitoring, and teamwork, enabling faster iterations and better outcomes in data science initiatives.
##### These principles are now being utilized in developing and managing AI-driven applications. As the demand for dependable and efficient AI systems grows, frameworks that enable smooth integration, tracking, and monitoring are becoming crucial. These frameworks ensure that AI applications deliver the same efficiency and reliability that DevOps has introduced to web development and MLOps has provided for data science projects. Langsmith plays a key role in achieving exactly this in the case of AI apps, and we will explore its capabilities in the next section.

# Integrating Langsmith into the LLM workflow

##### We'll create a Q&A system using RAG to answer user questions. For this example, we'll focus on geo-political news sourced from Wikipedia. The content will cover topics like G7 and G20 countries, their objectives, and other related information.

##### Here are the different stages we will implement in this section.
- Use the source content and generate Q&A
- Create a database on Langsmith and load the data
- Develop a function that processes the data and generates the output from the LLM
- Use the built-in evaluator from Langsmith and test for the accuracy of the answers from LLM.
- Trace all the steps and review the metrics.

![RAG Workflow](./LangSmith%20RAG.png)

### Load the libraries

In [ ]:
# !pip install langchain langchain_openai langchain_core

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import pandas as pd

### Project setup and API configurations

##### We used Google Colab and stored the API keys in the secrets. By setting the value  LANGCHAIN_TRACING_V2 to True, we enabled tracing with LangSmith. We will also define a project name that can be tracked on LangSmith.

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = "DEMO"

### Data processing

In [ ]:
context = """The G7 is a club of Western nations (with Japan given that status as an ally of the West and a major economy) that have dominated the world and its institutions, in some cases for centuries, and retain the ambition to maintain that position by policy coordination amongst themselves and by co-opting rising powers, including India, given the shifts in global power in recent decades.
The G7 recognised that they could not manage the 2008 financial crisis on their own and needed a wider international partnership, but one under their aegis. With this in mind, the G20 forum hitherto at the finance minister level was raised to the summit level. The G20 agenda is, however, shifting increasingly towards the interests and priorities of the developing countries (now being referred to as the Global South). During India’s G20 presidency, with India holding the Voice of the Global South summits before presiding over the G20 and at the conclusion of its work, and with the inclusion of the African Union as a G20 permanent member at India’s initiative, the pro-Global South content of the G20 agenda has got consolidated.
Both the G7 and the G20, however, face challenges from other platforms for consensus-building on global issues. BRICS, a group of non-Western countries, is getting expanded to resist the hegemony of the West that is still expressing itself in the form of sanctions, the weaponising of finance, regime change policies and double standards in addressing issues of democracy and human rights etc. An expanded BRICS will rival both the G7 and the G20 as a platform for promoting multipolarity, a greater role of developing countries in global governance, more equity in international relations, and introducing much-needed reforms in the international system."""


inputs = [
    "What is the G7, and how has it historically positioned itself in global governance?",
    "How did the 2008 financial crisis influence the role of the G20, and how has the agenda shifted?",
    "How has India influenced the G20 agenda during its presidency?",
    "What challenges do the G7 and G20 face from other global platforms?",
    "How does the expansion of BRICS pose a threat to the G7 and G20 in terms of global influence?"
]

outputs = [
    "The G7 is a group of Western nations, including Japan, which has historically dominated global institutions and policymaking. The G7's position stems from the economic and political power of its members, and they have coordinated policies to maintain their influence. In response to shifts in global power, the G7 has also sought to co-opt rising powers, such as India, in its strategic planning.",
    "The G7 recognized that it could not handle the 2008 financial crisis alone and needed broader international cooperation. As a result, the G20, which had previously operated at the finance minister level, was elevated to the summit level to ensure greater global participation under G7 guidance. Over time, the G20's agenda has shifted more towards the interests of developing countries, especially under India’s leadership, where pro-Global South priorities have become more prominent, including the inclusion of the African Union as a permanent member.",
    "During India’s G20 presidency, the country actively promoted the interests of developing countries by holding the 'Voice of the Global South' summits. India also pushed for the inclusion of the African Union as a permanent member of the G20, consolidating the agenda towards addressing the concerns of the Global South, such as greater equity and representation in global governance.",
    "Both the G7 and G20 face challenges from other groups like BRICS, which consists of non-Western countries seeking to resist Western dominance. BRICS has expanded as a counterbalance to the G7's influence, particularly criticizing Western sanctions, financial controls, and regime change policies. An expanded BRICS aims to promote multipolarity, increase the role of developing countries in global governance, and push for reforms in the international system.",
    "The expansion of BRICS is a direct challenge to the G7 and G20 as it aims to offer a platform that promotes multipolarity and reduces Western hegemony. By advocating for greater equity in international relations and pushing for reforms in global governance structures, an expanded BRICS seeks to rival the G7 and G20, providing an alternative consensus-building mechanism for developing nations and non-Western powers."
]

# Dataset
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)
df

,question,answer
0,"What is the G7, and how has it historically po...","The G7 is a group of Western nations, includin..."
1,How did the 2008 financial crisis influence th...,The G7 recognized that it could not handle the...
2,How has India influenced the G20 agenda during...,"During India’s G20 presidency, the country act..."
3,What challenges do the G7 and G20 face from ot...,Both the G7 and G20 face challenges from other...
4,How does the expansion of BRICS pose a threat ...,The expansion of BRICS is a direct challenge t...


### Creating a dataset on LangSmith

##### We'll take the data from the previous section to create a dataset in LangSmith. During this setup, we'll specify the dataset's name and provide a description for it as part of the configuration process.

In [ ]:
from langsmith import Client

client = Client()
dataset_name = "Geo-politics"

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about Geo-politics model.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

### Generating output from the LLM model

##### With the API keys, dataset, and other configurations in place, we can now create a function that processes the input question and generates a response using the LLM models - specifically, OpenAI in this case. The output of the function will be a dictionary.

In [ ]:
import openai
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(openai.Client())


def get_response_from_llm(inputs: dict) -> dict:
    """
    Generates answers to user questions based on a provided website text using OpenAI API.

    Parameters:
    inputs (dict): A dictionary with a single key 'question', representing the user's question as a string.

    Returns:
    dict: A dictionary with a single key 'output', containing the generated summary as a string.
    """

    # System prompt
    system_msg = (
         f"Answer user questions in 2-3 sentences about this context: \n\n\n {context}"
    )

    # Pass in website text
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": inputs["question"]},
    ]

    # Call OpenAI
    response = openai_client.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo"
    )

    # Response in output dict
    return {"answer": response.dict()["choices"][0]["message"]["content"]}

### Evaluating the RAG using the LLM model

##### We need to compare the LLM output with the ground truth, and there are various methods to do this. A common approach is using cosine similarity to measure how closely the LLM output matches the ground truth, with higher scores indicating more similarity. However, in this case, LangSmith offers a built-in evaluator, such as cot_qa, specifically designed for Q&A setups, which we'll use.

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# Evaluators
qa_evalulator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "Geo-politics"

experiment_results = evaluate(
    get_response_from_llm,
    data=dataset_name,
    evaluators=qa_evalulator,
    experiment_prefix="LLM Ouput",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "stuff website context into gpt-3.5-turbo",
    },
)

# Conclusion

##### As AI technology continues to advance, the importance of tools like Langsmith cannot be overstated. They help developers and data scientists collaborate effectively, enhancing the overall efficiency and reliability of AI systems. By embracing Langsmith, organizations can ensure their applications deliver exceptional user experiences while meeting the growing demands of the market.With Langsmith at your side, you can navigate the complexities of AI development with confidence. As we move forward in this exciting field, the insights and capabilities provided by Langsmith will be crucial in unlocking the full potential of your AI applications. Embrace this journey and watch your LLM-driven systems thrive!